# Answering Business Questions using SQL

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Overview of the Data

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Selecting Albums to Purchase

In [3]:
%%sql

WITH usa_tracks_sold AS
    (
     SELECT il.* FROM invoice_line il
     INNER JOIN invoice i on i.invoice_id=il.invoice_id
     INNER JOIN customer c ON c.customer_id=i.customer_id
     WHERE c.country="USA"   
    )
    
SELECT g.name genre,
       COUNT(uts.invoice_line_id) tracks_sold,
       CAST(COUNT(uts.invoice_line_id) AS FLOAT)/(SELECT COUNT(*)
                                                 FROM usa_tracks_sold) percentage_sold
FROM usa_tracks_sold uts
INNER JOIN track t ON t.track_id=uts.track_id
INNER JOIN genre g ON g.genre_id=t.genre_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;

Done.


genre,tracks_sold,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


Based on the data of sales on track across different genre in the US.One must purchase albums of following artists:
- Red Tone(Punk)
- Slim Jim Bites(Blues)
- Meteor and the Girls(PUnk)

Anyway rock is the popular genre controling the whole music genre with sales more than half of the whole so in futire rock must be given substantial preference.

## Analyzing Employee Sales Performance

In [4]:
%%sql

WITH customer_support_id_sale AS
    (
     SELECT
           i.customer_id,
           c.support_rep_id,
           SUM(i.total) total
     FROM invoice i
     INNER JOIN customer c ON c.customer_id=i.customer_id
     GROUP BY 1
    )
SELECT e.first_name || " " || e.last_name employee_name,
       e.hire_date,
       SUM(csis.total) total_sales
FROM customer_support_id_sale csis
INNER JOIN employee e ON e.employee_id=csis.support_rep_id
GROUP BY 1;

Done.


employee_name,hire_date,total_sales
Jane Peacock,2017-04-01 00:00:00,1731.5099999999998
Margaret Park,2017-05-03 00:00:00,1584.0000000000002
Steve Johnson,2017-10-17 00:00:00,1393.92


Jane has almost 20% high sales than Steve which corresponds to only arising due to the difference in their hiring dates

## Analyzing Sales by Country

In [5]:
%%sql
SELECT country,COUNT(*)
FROM customer
GROUP BY 1;

Done.


country,COUNT(*)
Argentina,1
Australia,1
Austria,1
Belgium,1
Brazil,5
Canada,8
Chile,1
Czech Republic,2
Denmark,1
Finland,1


In [6]:
%%sql
SELECT * 
FROM invoice_line il 
LIMIT 5;



Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
1,1,1158,0.99,1
2,1,1159,0.99,1
3,1,1160,0.99,1
4,1,1161,0.99,1
5,1,1162,0.99,1


In [7]:
%%sql
SELECT *
FROM invoice_line il
INNER JOIN invoice i ON i.invoice_id=il.invoice_id
LIMIT 5;

Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity,invoice_id_1,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,1,1158,0.99,1,1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,1,1159,0.99,1,1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
3,1,1160,0.99,1,1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
4,1,1161,0.99,1,1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
5,1,1162,0.99,1,1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84


In [8]:
%%sql 
SELECT
      CASE
          WHEN (
                 SELECT COUNT(*)
                 FROM customer
                 WHERE country=c.country)=1 THEN 'Others'
          ELSE c.country
     END AS country,
      c.customer_id,
      il.*
FROM invoice_line il
INNER JOIN invoice i ON i.invoice_id=il.invoice_id
INNER JOIN customer c ON c.customer_id=i.customer_id
LIMIT 10;


Done.


country,customer_id,invoice_line_id,invoice_id,track_id,unit_price,quantity
USA,18,1,1,1158,0.99,1
USA,18,2,1,1159,0.99,1
USA,18,3,1,1160,0.99,1
USA,18,4,1,1161,0.99,1
USA,18,5,1,1162,0.99,1
USA,18,6,1,1163,0.99,1
USA,18,7,1,1164,0.99,1
USA,18,8,1,1165,0.99,1
USA,18,9,1,1166,0.99,1
USA,18,10,1,1167,0.99,1


In [9]:
%%sql

WITH country_or_other AS
(
 SELECT
   CASE
       WHEN (SELECT COUNT(*)
             FROM customer
             WHERE country=c.country)=1 THEN 'Other'
       ELSE c.country
   END AS country,
   c.customer_id,
   il.*
 FROM invoice_line il
 INNER JOIN invoice i ON i.invoice_id=il.invoice_id
 INNER JOIN customer c ON c.customer_id=i.customer_id
)
SELECT 
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        COUNT(DISTINCT customer_id) customers,
        SUM(unit_price) total_sales,
        SUM(unit_price)/COUNT(DISTINCT customer_id) customer_lifetime_value,
        SUM(unit_price)/COUNT(DISTINCT invoice_id) average_order,
        CASE
            WHEN country='Other' THEN 1
            ELSE 0
        END AS sort
     FROM country_or_other
     GROUP BY country
     ORDER BY sort ASC
    );
    

Done.


country,customers,total_sales,average_order,customer_lifetime_value
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
Other,15,1094.9400000000085,7.448571428571486,72.99600000000056
